# Mercor Trial Task - GitHub Recommendation 


## Importing libraries

In [2]:
!pip install -qU bs4 tiktoken openai langchain pinecone-client

In [24]:
!pip uninstall pinecone-client -y

Found existing installation: pinecone-client 2.2.4
Uninstalling pinecone-client-2.2.4:
  Successfully uninstalled pinecone-client-2.2.4


In [1]:
!pip install 'pinecone-client[grpc]'

## Preparing the Data

### I had stored the GitHub Parsed data in a JSON file. Reading the parsed data from a JSON file.

In [108]:
#Read My Custom Data
import json 

json_data = None
def get_json_data():
    # Open and read the JSON file
    with open('restructure_json_text.json', 'r') as json_file:
        data = json.load(json_file)
    return data

json_data_text = get_json_data()

In [109]:
custom_data = json_data_text['data']

In [110]:
custom_data

[{'repo_name': 'API_Development',
  'text': 'sourceurl  httpspypiorgsimpleverify_ssl  truename  pypipackagesdevpackagesrequirespython_version  310 market analysis systemthe market analysis system is a backend api returns a analytical data from server using 4 apisthe system provides four apis1 api 1end point  apitotal_items api use cases 1 total item total seats sold in marketting for last in q3 of the yearexpected op returns integerparameters start_date date end_date date department string2 api 2end point  apinth_most_total_item api use cases1what is the 2nd most sold item in terms of quantity sold in q42what is the fourth most sold item in terms of total price in q2expected op returns string nameparameters  item_by quantity   price start_date date end_datedate ninteger 3 api 3end point  apipercentage_of_department_wise_sold_itemsapi use cases1what is the percentage of sold items seats department wiseexpected op dept_name x parameters start_date date end_date date4 api 4end point  apim

Using `langchain` and `tiktoken` to create chunks of textual data since there is token limit:

In [111]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [112]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [113]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(custom_data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'repo_name': record['repo_name']
    } for i in range(len(texts))])

  0%|          | 0/3 [00:00<?, ?it/s]

## Initialize Embedding Model

We use `text-embedding-ada-002` as the embedding model.

In [114]:
import openai

# initialize openai API key
openai.api_key = "sk-..."  #platform.openai.com

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In [115]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Each vector embedding contains `1536` dimensions (the output dimensionality of the `text-embedding-ada-002` model.

In [117]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

## Initializing the Index

To store these embeddings and enable an efficient vector search through them I used Pinecone. 

In [106]:
pinecone.delete_index("trial")

In [118]:
import pinecone

index_name = 'trial-1'

# initialize connection to pinecone
pinecone.init(
    api_key="pine_cone_key",  # app.pinecone.io (console)
    environment="gcp-starter"  # next to API key in console
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='cosine'
    )
# connect to index
index = pinecone.Index(index_name)
# index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [119]:
pinecone.list_indexes()

['trial-1']

### Computing embeddings and uploading them to the index for a batch

In [122]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'repo_name': x['repo_name']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/6 [00:00<?, ?it/s]

In [124]:
index.fetch(['84235634-2259-4899-a3fa-8c1e2d17e818'])

{'namespace': '',
 'vectors': {'84235634-2259-4899-a3fa-8c1e2d17e818': {'id': '84235634-2259-4899-a3fa-8c1e2d17e818',
                                                      'metadata': {'chunk': 500.0,
                                                                   'repo_name': 'API_Development',
                                                                   'text': '004229 '
                                                                           '0530dr '
                                                                           'vm '
                                                                           'koothrappalimarkettingzoho '
                                                                           'crm3540921720220713 '
                                                                           '225138 '
                                                                           '0530barry '
                                                                         

## Retrieval

Create a query vector to search through our vector db. We will retrieve the most relevant chunks from the GitHub parsed data. 

In [179]:
query = "give me repository that uses celery framework"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [180]:
res

{'matches': [{'id': 'b1bb9b4c-dd64-4b86-90e9-c685a64ea1f9',
              'metadata': {'chunk': 4.0,
                           'repo_name': 'Restaurant-poller',
                           'text': 'functionalities used advance python '
                                   'features like   rabbitmq queue for '
                                   'generating report  celery worker  '
                                   'multithreading  cachingdefault_app_config  '
                                   'apiappsapiconfigfrom djangocontrib import '
                                   'admin register your models herefrom '
                                   'djangoapps import appconfigimport '
                                   'threadingimport django from djangoconf '
                                   'import settingsclass apiconfigappconfig    '
                                   'default_auto_field  '
                                   'djangodbmodelsbigautofield    name  '
                      

### Defined a method to extract relevant info from the response such as id, chunk, repo_name and text. I convert it into specific format and convert it into string so that I can combine it with the original query, and use GPT4 to get reasoning for choosing the top k chunks. 

In [181]:
import json

def transform_data_to_string(data):
    transformed_list = []
    
    matches = data.get('matches', [])
    
    for match in matches:
        transformed_dict = {}
        
        transformed_dict['id'] = match.get('id')
        transformed_dict['chunk'] = match.get('metadata').get('chunk')
        transformed_dict['repo_name'] = match.get('metadata').get('repo_name')
        transformed_dict['text'] = match.get('metadata').get('text')
        
        transformed_list.append(transformed_dict)
        
    # Convert the list of dictionaries to a JSON-formatted string
    return json.dumps(transformed_list)

In [182]:
transformed_data_str = transform_data_to_string(res)

In [183]:
transformed_data_str

'[{"id": "b1bb9b4c-dd64-4b86-90e9-c685a64ea1f9", "chunk": 4.0, "repo_name": "Restaurant-poller", "text": "functionalities used advance python features like   rabbitmq queue for generating report  celery worker  multithreading  cachingdefault_app_config  apiappsapiconfigfrom djangocontrib import admin register your models herefrom djangoapps import appconfigimport threadingimport django from djangoconf import settingsclass apiconfigappconfig    default_auto_field  djangodbmodelsbigautofield    name  api        def readyself        pass        from service import run_import_data        from apiservice import refresh_cache             t  threadingthreadtargetrefresh_cache        tstart generated by django 421 on 20230601 1742from djangodb import migrations modelsclass migrationmigrationsmigration    initial  true    dependencies          operations          migrationscreatemodel            namebuissnesshour            fields                id modelsbigautofieldauto_createdtrue primary_key

## Retrieval Augmented Generation

So far we have the top k results (chunks) that are relevant to the input query. To generate 2-3 sentences regarding why they are relevant, I used gpt4. I input the extracted results along with the query. I also define a query template, to give a better context to the system about the task that needs to be performed by the gpt4 system.

In [171]:
augmented_query = "{ 'Query':'" + query + "'," + "'chunk_list':'" + transformed_data_str + "'}"

In [172]:
print(augmented_query)

{ 'Query':'give me repository that uses celery framework','chunk_list':'[{"id": "b1bb9b4c-dd64-4b86-90e9-c685a64ea1f9", "chunk": 4.0, "repo_name": "Restaurant-poller", "text": "functionalities used advance python features like   rabbitmq queue for generating report  celery worker  multithreading  cachingdefault_app_config  apiappsapiconfigfrom djangocontrib import admin register your models herefrom djangoapps import appconfigimport threadingimport django from djangoconf import settingsclass apiconfigappconfig    default_auto_field  djangodbmodelsbigautofield    name  api        def readyself        pass        from service import run_import_data        from apiservice import refresh_cache             t  threadingthreadtargetrefresh_cache        tstart generated by django 421 on 20230601 1742from djangodb import migrations modelsclass migrationmigrationsmigration    initial  true    dependencies          operations          migrationscreatemodel            namebuissnesshour            

In [177]:
# system message to 'prime' the model
primer = """ You are a highly intelligent content retrieval system that answers
user queries. Think step by step and thoughtfully. You need to refer to 'chunk_list' and for each of the chunk data, refer to the 'text' key and provide a brief three-sentence explanation detailing why that particular text is relevant to the query mentioned in 'Query'.
Retrieve and display a 1-2 sentence preferably of Python code and do not return plain text that is not Python code from each chunk data (content is parsed from Github repositories) that best matches the provided query.
Your answer should be in the format: 
[{"id_1":, "summary_1":, "relevant_text_1":},{"id_2":, "summary_2":,"relevant_text_2":}]
If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

In [178]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

[{"id_1": "b1bb9b4c-dd64-4b86-90e9-c685a64ea1f9", "summary_1": "This excerpt showcases the use of advanced Python features in the 'Restaurant-poller' repository, such as using the Celery worker for multithreading and a RabbitMQ queue for generating a report.", "relevant_text_1": "from service import run_import_data\nfrom apiservice import refresh_cache\n\nt = threading.thread(target=refresh_cache)\nt.start()"},
{"id_2": "1b86e977-d8e9-4627-8960-c22605454bfe", "summary_2": "The excerpt from the same repository demonstrates how to set up the environment with RabbitMQ and PostgreSQL docker containers and how to run a Celery worker to start the task queue.", "relevant_text_2": "docker run -d --hostname myrabbit --name restaurent-manager rabbitmq:3-management\nc_force_root=1 celery -A config worker -l info -P solo -Q generate_report -n config@%h"},
{"id_3": "1be12c0b-2782-4407-aab5-7f2de5aef342", "summary_3":"In this text from same repository, it lists all the installed packages that includes Celery version 5.2.7 noting the usage of Celery framework.", "relevant_text_3":"celery==5.2.7"}]